# Импорты библиотек

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as st
from datetime import datetime

# Чтение данных

In [ ]:
df_path = "marketplace.csv"

df = pd.read_csv(df_path)
display(df.head())
df.info()

# Чистка данных

Очищенные данные кладем в переменную df_cleaned. 

После очистки запишем данные в новый файл marketplace_cleaned.csv для быстрого доступа к ним.
1. Преобразовать типов
1. Дубли юзеров сливаем в одну запись суммируя покупки и возвраты, таргет - среднее
1. Формат данных - юзеров делаем индексами
1. Заполнить пустые даты. Отбросить выбросы. Взять среднее кол-во дней между датой регистрации и датой покупки. Заполнить пустые даты регистрации вычитая среднее кол-во дней между покупкой и регистрацией.


## Преобразование типов

In [ ]:
df_cleaned = pd.DataFrame(data=df)

df_cleaned["reg_dt"] = pd.to_datetime(df_cleaned["reg_dt"], errors="coerce")
df_cleaned["first_buy"] = pd.to_datetime(df_cleaned["first_buy"], errors="coerce")
df_cleaned["first_login"] = pd.to_datetime(df_cleaned["first_login"], errors="coerce")

df_cleaned["user_id"] = df_cleaned["user_id"].apply(lambda x: int(x.strip("user_")))
df_cleaned["browser"] = df_cleaned["browser"].apply(lambda x: int(x.strip("browser_")))

## Заполнение нулей

In [ ]:
df_cleaned["first_buy"].fillna(df_cleaned[['first_login', 'reg_dt']].max(axis=1), inplace=True)

## Склеивание дублей и индексация по user_id

In [ ]:
df_cleaned = df_cleaned.groupby(by="user_id").agg(
    {
        "platform_num": "min",
        "first_login": "min",
        "reg_dt": "min",
        "browser": "min",
        "first_buy": "min",
        "target": "mean",
        "total_buy": "sum",
        "total_return": "sum",
    }
)
print(len(df_cleaned))
df_cleaned.head()

In [ ]:
df_cleaned.loc[(df_cleaned['reg_dt'] > df_cleaned['first_buy']), ['reg_dt']] = df_cleaned['first_buy']
df_cleaned.loc[(df_cleaned['first_login'] > df_cleaned['first_buy']), ['first_login']] = df_cleaned['first_buy']

## Проверить на нормальность распределения

## Проверить равенство дисперсий

## Посчитать межквартильный размах

## Посчитать zscore

## Убрать выбросы по межквартильному размаху

## Убрать выбросы по zscore

In [ ]:
df_zcore_filtered = pd.DataFrame(data=df_cleaned)

df_zcore_filtered = df_zcore_filtered[
    (abs(st.zscore(df_cleaned["total_buy"])) < 3)
    & (abs(st.zscore(df_cleaned["total_return"])) < 3)
]
print(len(df_zcore_filtered))
df_zcore_filtered.head()

## Наблюдаемые ошибки в датах. Даты не соответствуют порядку: дата первого логина -> дата первой покупки

In [ ]:
def difference(dataset, first_dt, second_dt):
    return dataset[first_dt] - dataset[second_dt]

print('\nBefore cleaning:')
print(f'Первая покупка раньше регистрации у {len(df.loc[(df['reg_dt'] > df['first_buy'])])} из {len(df)}')
print(f'Первая покупка раньше первого логина у {len(df.loc[(df['first_login'] > df['first_buy'])])} из {len(df)}')
print(f'Оба условия (или) у {len(df.loc[(df['first_login'] > df['first_buy']) | (df['reg_dt'] > df['first_buy'])])} из {len(df)}')

print('\nAfter cleaning:')
print(f'Первая покупка раньше регистрации у {len(df_cleaned.loc[(df_cleaned['reg_dt'] > df_cleaned['first_buy'])])} из {len(df_cleaned)}')
print(f'Первая покупка раньше первого логина у {len(df_cleaned.loc[(df_cleaned['first_login'] > df_cleaned['first_buy'])])} из {len(df_cleaned)}')
print(f'Средняя разница дат - {difference(df_cleaned.loc[(df_cleaned['first_login'] > df_cleaned['first_buy'])], 'first_login', 'first_buy').mean()}')
print(f'Оба условия (или) у {len(df_cleaned.loc[(df_cleaned['first_login'] > df_cleaned['first_buy']) | (df_cleaned['reg_dt'] > df_cleaned['first_buy'])])} из {len(df_cleaned)}')

print('\nAfter cleaning without outlines:')
print(f'Первая покупка раньше регистрации у {len(df_zcore_filtered.loc[(df_zcore_filtered['reg_dt'] > df_zcore_filtered['first_buy'])])} из {len(df_zcore_filtered)}')
print(f'Первая покупка раньше первого логина у {len(df_zcore_filtered.loc[(df_zcore_filtered['first_login'] > df_zcore_filtered['first_buy'])])} из {len(df_zcore_filtered)}')
print(f'Оба условия (или) у {len(df_zcore_filtered.loc[(df_zcore_filtered['first_login'] > df_zcore_filtered['first_buy']) | (df_zcore_filtered['reg_dt'] > df_zcore_filtered['first_buy'])])} из {len(df_zcore_filtered)}')

## Повторить проверки на нормальность и равество дисперсий после исключения выбросов по межквартильному размаху

## Повторить проверки на нормальность и равество дисперсий после исключения выбросов по zscore

In [ ]:
# fig, ax = plt.subplots(figsize=(16, 6))

# sns.scatterplot(data=df_zcore_filtered, x="browser", y="target", hue="total_buy")

# plt.show()

In [ ]:
# Импульсивные покупки
# sns.scatterplot(data=df_zcore_filtered, x="first_login", y="first_buy", hue="total_buy")


Добавим в датафрейм день недели и день месяца

In [ ]:
# fig, ax = plt.subplots(figsize=(16, 6))
# df_zcore_filtered["reg_wday"] = df_zcore_filtered["reg_dt"].apply(lambda x: x.weekday())
# df_zcore_filtered["login_wday"] = df_zcore_filtered["first_login"].apply(lambda x: x.weekday())
# df_zcore_filtered["buy_wday"] = df_zcore_filtered["first_buy"].apply(
#     lambda x: x.weekday()
# )
# df_zcore_filtered["reg_mday"] = df_zcore_filtered["reg_dt"].apply(lambda x: x.day)
# df_zcore_filtered["login_mday"] = df_zcore_filtered["first_login"].apply(lambda x: x.day)
# df_zcore_filtered["buy_mday"] = df_zcore_filtered["first_buy"].apply(
#     lambda x: x.day
# )

# sns.scatterplot(
#     data=df_zcore_filtered,
#     x="login_mday",
#     y="total_buy",
#     hue="buy_wday",
# )

# plt.show()

# Гипотезы

### Рассмотрим следующие гипотезы:
 1. Влияние заранее зарегистрировавшихся пользователей (задолго до первого логина/покупки) и пользователей, которые произвели покупку до регистрации
 2. Таргет может влиять на покупки или наоборот
 3. Влияние браузера на прибыльность или на скорость покупки.
 4. Рассмотреть превалирирование определённого браузера/браузеров у юзеров, совершивших больше всего возвратов.
 5. Аномальные всплески покупок/возвратов за весь период анализа. Привязка к другим данным? Праздникам?

# Стратегия проверки: Строим графики, смотрим результаты, считаем. Думаем. Повторяем. (кто-нибудь помогите с формулировкой, пожалуйста)


### Гипотеза №1 "Консервы"
_Влияние заранее зарегистрировавшихся пользователей и пользователей, которые произвели покупку до регистрации_

На выдвижение данной гипотезы меня натолкнула идея построения графиков зависимостей всех трёх дат (регистрация, первый логин, первая покупка) попарно, в поисках аномалий.

In [ ]:
# обогащу таблицу разницей времени между тремя событиями
df_enriched = df_zcore_filtered.copy()

df_enriched["first_buy_reg_diff"] = (
    df_zcore_filtered["first_buy"] - df_zcore_filtered["reg_dt"]
).dt.days
df_enriched["first_login_buy_diff"] = (
    df_zcore_filtered["first_login"] - df_zcore_filtered["first_buy"]
).dt.days
df_enriched["first_login_reg_diff"] = (
    df_zcore_filtered["first_login"] - df_zcore_filtered["reg_dt"]
).dt.days
display(df_enriched.head(2))


In [ ]:
# Строим график зависимости дат регистрации и дат первой покупки
plt.figure(figsize=(10, 6))

# Рисуем точки
plt.scatter(df_enriched["reg_dt"], df_enriched["first_buy"], alpha=0.5, s=10)

# Добавляем линию "мгновенной покупки" (где X=Y), чтобы видеть задержку
# (для наглядности просто проведем диагональ)
min_date = df_enriched["reg_dt"].min()
max_date = df_enriched["first_buy"].max()
plt.plot(
    [min_date, max_date],
    [min_date, max_date],
    color="red",
    linestyle="--",
    label="Моментальная покупка",
)

plt.title("Когда регистрировались vs Когда купили")
plt.xlabel("Дата регистрации")
plt.ylabel("Дата первой покупки")
plt.legend()
plt.grid(True)
plt.show()


Видим аномальные отклонения от предсказуемой красной линии, есть объёмная группа юзеров, что регестрировались на протяжении всего периода, но покупки совершали в марте-апреле 2025. Это требует дополнительных исследований.

In [ ]:
# Строим график зависимости дат первого логина и дат первой покупки
plt.figure(figsize=(10, 6))

# Рисуем точки
plt.scatter(df_enriched["first_login"], df_enriched["first_buy"], alpha=0.5, s=10)

# Добавляем линию "мгновенной покупки" (где X=Y), чтобы видеть задержку
# (для наглядности просто проведем диагональ)
min_date = df_enriched["first_login"].min()
max_date = df_enriched["first_buy"].max()
plt.plot(
    [min_date, max_date],
    [min_date, max_date],
    color="red",
    linestyle="--",
    label="Моментальная покупка",
)

plt.title("Когда первый логин vs Когда купили")
plt.xlabel("Дата первого логина")
plt.ylabel("Дата первой покупки")
plt.legend()
plt.grid(True)
plt.show()


Тут, предварительно, не выводится надёжной визуальной зависимости. Вертикальная разряженная линия появилась из-за заполнения значений first_buy от reg_dt, который не учавствует в данном графике.
Это, дополнительно, проверяется отстройкой подобного графика на неочищенных данных:
# Возможно лишний график стоит убрать, он вызовет вопросы, а я отвечаю на то, о чём нас и не спрашивали.
![image.png](attachment:image.png)

In [ ]:
# Строим график зависимости дат регистрации и дат первого логина
plt.figure(figsize=(10, 6))

# Рисуем точки
plt.scatter(df_enriched["reg_dt"], df_enriched["first_login"], alpha=0.5, s=10)

# Добавляем линию "мгновенной покупки" (где X=Y), чтобы видеть задержку
# (для наглядности просто проведем диагональ)
min_date = df_enriched["reg_dt"].min()
max_date = df_enriched["first_login"].max()
plt.plot(
    [min_date, max_date],
    [min_date, max_date],
    color="red",
    linestyle="--",
    label="Моментальная покупка",
)

plt.title("Когда регистрировались vs Когда первый логин")
plt.xlabel("Дата регистрации")
plt.ylabel("Дата первого логина")
plt.legend()
plt.grid(True)
plt.show()


Видим аномалию, аналогичную первому графику, соответственно регистрация была давно, а первая покупка и логин (если это одни и те же пользователи) произведены в марте-апреле 2025.
Также заметен дополнительный аномальный интервал к концу 2024 года, также требующий анализа.

Для проведения анализа планируется выборка клиентов с "лагом" в датах более ~30-50 дней и сравнение их финансового вклада по сравнению с остальными пользователями.

### Гипотеза №4 "Браузерные войны!"
_Предпочтения браузера у юзеров, совершивших больше всего покупок/возвратов._

Рассмотрим корреляцию используемых браузеров юзерами.

In [ ]:
# Укажите список + стратегию проверки